# Run the API on the 2022_2023 School Student Addresses
## Goal
Translate the Student Addresses into valid queries that can be given to an API and exchanged with coordinates for plot-able data points.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

# Get the file and set to a variable named filename
filename = os.path.join(os.getcwd(), "/content/drive/MyDrive/CS506_Project/API/Altered_datasets", "cleaned_student_addresses_2022_2023.csv")

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(filename, header = 0)

In [ ]:
def clean_zipcode(zip_code):
    if pd.isna(zip_code):
        return zip_code # Handle NaN values
    s_zip = str(zip_code).strip()

    # If the zip code contains a hyphen, take only the first part
    if '-' in s_zip:
        s_zip = s_zip.split('-')[0]

    # Handle cases like "002134" which should become "02134"
    if len(s_zip) == 6 and s_zip.startswith('00'):
        return s_zip[1:]

    return s_zip

# Apply the cleaning function to the '6e. zip' column
df['6e. zip'] = df['6e. zip'].apply(clean_zipcode)

In [ ]:
df.head(50)

,6a. street #,6b. street name,6c. street suffix,6e. zip,7. undergraduate (u) or graduate (g),8. full-time (ft) or part-time (pt),9. at-home or not-at-home,university,year,Street_combined,City,State
0,1235,Vfw,Parkway,02132,U,FT,At Home,Bay State College,2022-2023,1235 Vfw Parkway,Boston,Massachusetts
1,7,Aberdeen,Street,02215,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,7 Aberdeen St,Boston,Massachusetts
2,17,Aberdeen,Street,02215,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,17 Aberdeen St,Boston,Massachusetts
3,144,Addison,Street,02128,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,144 Addison St,Boston,Massachusetts
4,601,Albany,Street,02118,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,601 Albany St,Boston,Massachusetts
5,12,Albemarle,Court,02115,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,12 Albemarle Ct,Boston,Massachusetts
6,20,Alcott,Street,02134,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,20 Alcott St,Boston,Massachusetts
7,15,Aldie,Street,02134,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,15 Aldie St,Boston,Massachusetts
8,23,Aldie,Street,02134,G,PT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,23 Aldie St,Boston,Massachusetts
9,37,Aldie,Street,02134,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,37 Aldie St,Boston,Massachusetts


## Formatting for the API Query functions needed
Use of Copilot to create the corresponding code to help translate the columns that we have into usable strings that can be compiled into a valid url to query the api.

In [ ]:
import time
import requests
import pandas as pd
import numpy as np
from typing import Optional, Tuple, Dict

GEOCODE_URL = "https://geocode.maps.co/search"

def _build_address_string(street: Optional[str], city: Optional[str],
                          state: Optional[str], postalcode: Optional[str]) -> str:
    parts = []
    if street and str(street).strip():
        parts.append(str(street).strip())
    if city and str(city).strip():
        parts.append(str(city).strip())
    if state and str(state).strip():
        parts.append(str(state).strip())
    if postalcode and str(postalcode).strip():
        parts.append(str(postalcode).strip())
    return ", ".join(parts)

def geocode_single(session: requests.Session,
                    street: Optional[str],
                    city: Optional[str],
                    state: Optional[str],
                    postalcode: Optional[str],
                    api_key: Optional[str] = None,
                    country: Optional[str] = "US",
                    timeout: float = 10.0,
                    max_retries: int = 4,
                    pause: float = 1.0) -> Tuple[Optional[float], Optional[float]]:
    """
    Geocode a single address. Returns (lat, lon) or (None, None) if no result.
    Uses exponential backoff for 429/503 responses.
    """
    address_str = _build_address_string(street, city, state, postalcode)
    if not address_str:
        return None, None

    params = {
        "format": "json",
        "limit": 1
    }
    # Prefer structured search: include components when available
    if street:
        params["street"] = street
    if city:
        params["city"] = city
    if state:
        params["state"] = state
    if postalcode:
        params["postalcode"] = postalcode
    if country:
        params["country"] = country

    # If user prefers to pass api_key as query param, uncomment next line
    # if api_key: params["api_key"] = api_key

    headers = {
        "User-Agent": "geocode-client/1.0"
    }
    if api_key:
        headers["Authorization"] = f"Bearer {api_key}"

    backoff = pause
    for attempt in range(1, max_retries + 1):
        try:
            resp = session.get(GEOCODE_URL, params=params, headers=headers, timeout=timeout)
        except requests.RequestException:
            # network failure
            if attempt == max_retries:
                return None, None
            time.sleep(backoff)
            backoff *= 2
            continue

        if resp.status_code == 200:
            try:
                data = resp.json()
            except ValueError:
                return None, None
            if isinstance(data, list) and len(data) > 0:
                first = data[0]
                try:
                    lat = float(first.get("lat"))
                    lon = float(first.get("lon"))
                    return lat, lon
                except Exception:
                    return None, None
            else:
                # no matches
                return None, None

        if resp.status_code in (429, 503):
            # rate limited or service unavailable - backoff and retry
            if attempt == max_retries:
                return None, None
            time.sleep(backoff)
            backoff *= 2
            continue

        # for 403/4xx/5xx other than 429/503 -> don't retry too aggressively
        # if 403, you might be blocked/need to contact API provider
        return None, None

    return None, None


def geocode_addresses(df: pd.DataFrame,
                      street_col: str,
                      city_col: Optional[str],
                      state_col: Optional[str],
                      postal_col: Optional[str],
                      api_key: Optional[str] = None,
                      country: str = "US",
                      timeout: float = 10.0,
                      pause: float = 1.0,
                      max_retries: int = 4,
                      max_rows: Optional[int] = None,
                      show_progress: bool = True) -> pd.DataFrame:
    """
    Geocode addresses from a DataFrame and return a copy with 'latitude_geocoded'
    and 'longitude_geocoded' columns appended.

    Example:
      df2 = geocode_addresses(df, "street_col", "city_col", "state_col", "zip_col", api_key="XXX")
      # then df2 has new columns you can assign back: df["lat"] = df2["latitude_geocoded"]
    """
    session = requests.Session()
    cache: Dict[str, Tuple[Optional[float], Optional[float]]] = {}

    n = len(df) if max_rows is None else min(len(df), max_rows)
    latitudes = []
    longitudes = []

    it = range(n)
    for i in it:
        if show_progress and (i % 50 == 0):
            print(f"Geocoding row {i+1}/{n} ...")
        row = df.iloc[i]
        street = row.get(street_col, None)
        city = row.get(city_col, None) if city_col else None
        state = row.get(state_col, None) if state_col else None
        postal = row.get(postal_col, None) if postal_col else None

        addr_key = _build_address_string(street, city, state, postal)
        if addr_key in cache:
            lat, lon = cache[addr_key]
        else:
            lat, lon = geocode_single(session,
                                      street=street,
                                      city=city,
                                      state=state,
                                      postalcode=postal,
                                      api_key=api_key,
                                      country=country,
                                      timeout=timeout,
                                      max_retries=max_retries,
                                      pause=pause)
            cache[addr_key] = (lat, lon)
            # polite pause between requests to avoid being rate-limited
            time.sleep(pause)

        latitudes.append(lat if lat is not None else np.nan)
        longitudes.append(lon if lon is not None else np.nan)

    # Build output DataFrame (copy to avoid mutating input)
    out = df.copy()
    # New columns are aligned with the subset length; if max_rows set, fill rest with NaN
    if max_rows is not None and max_rows < len(df):
        # create full-length lists
        lat_full = [np.nan] * len(df)
        lon_full = [np.nan] * len(df)
        for j in range(max_rows):
            lat_full[j] = latitudes[j]
            lon_full[j] = longitudes[j]
        out["latitude_geocoded"] = lat_full
        out["longitude_geocoded"] = lon_full
    else:
        out["latitude_geocoded"] = latitudes + [np.nan] * (len(df) - len(latitudes))
        out["longitude_geocoded"] = longitudes + [np.nan] * (len(df) - len(longitudes))

    return out

In [ ]:
df_test = geocode_addresses(df, "Street_combined", "City", "State", "6e. zip", "INSERTKEY")

display(df_test)

Geocoding row 1/11058 ...
Geocoding row 51/11058 ...
Geocoding row 101/11058 ...
Geocoding row 151/11058 ...
Geocoding row 201/11058 ...
Geocoding row 251/11058 ...
Geocoding row 301/11058 ...
Geocoding row 351/11058 ...
Geocoding row 401/11058 ...
Geocoding row 451/11058 ...
Geocoding row 501/11058 ...
Geocoding row 551/11058 ...
Geocoding row 601/11058 ...
Geocoding row 651/11058 ...
Geocoding row 701/11058 ...
Geocoding row 751/11058 ...
Geocoding row 801/11058 ...
Geocoding row 851/11058 ...
Geocoding row 901/11058 ...
Geocoding row 951/11058 ...
Geocoding row 1001/11058 ...
Geocoding row 1051/11058 ...
Geocoding row 1101/11058 ...
Geocoding row 1151/11058 ...
Geocoding row 1201/11058 ...
Geocoding row 1251/11058 ...
Geocoding row 1301/11058 ...
Geocoding row 1351/11058 ...
Geocoding row 1401/11058 ...
Geocoding row 1451/11058 ...
Geocoding row 1501/11058 ...
Geocoding row 1551/11058 ...
Geocoding row 1601/11058 ...
Geocoding row 1651/11058 ...
Geocoding row 1701/11058 ...
Geocodin

,6a. street #,6b. street name,6c. street suffix,6e. zip,7. undergraduate (u) or graduate (g),8. full-time (ft) or part-time (pt),9. at-home or not-at-home,university,year,Street_combined,City,State,latitude_geocoded,longitude_geocoded
0,1235,Vfw,Parkway,02132,U,FT,At Home,Bay State College,2022-2023,1235 Vfw Parkway,Boston,Massachusetts,42.278976,-71.171238
1,7,Aberdeen,Street,02215,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,7 Aberdeen St,Boston,Massachusetts,42.346305,-71.103884
2,17,Aberdeen,Street,02215,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,17 Aberdeen St,Boston,Massachusetts,42.346156,-71.103808
3,144,Addison,Street,02128,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,144 Addison St,Boston,Massachusetts,42.385526,-71.014492
4,601,Albany,Street,02118,U,FT,NaN,Berklee College of Music-Boston Conservatory,2022-2023,601 Albany St,Boston,Massachusetts,42.337167,-71.068785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11053,28,Senator Bolling,Circle,02124,G,Full Time,Home,Boston College,2022-2023,28 Senator Bolling Circle,Boston,Massachusetts,42.290909,-71.092921
11054,27,Tafthill,Terrace,02131,G,Full Time,Home,Boston College,2022-2023,27 Tafthill Terrace,Boston,Massachusetts,42.287884,-71.127589
11055,944,Vfw,Parkway,02132,G,Full Time,Home,Boston College,2022-2023,NaN,Boston,Massachusetts,42.354992,-71.134306
11056,736,West Roxbury,Parkway,02131,G,Part Time,Home,Boston College,2022-2023,NaN,Boston,Massachusetts,42.354992,-71.134306


In [ ]:
# Count NaN values in each column
nan_counts = df_test.isnull().sum()

print("NaN counts per column:")
print(nan_counts)

NaN counts per column:
6a. street #                             115
6b. street name                            0
6c. street suffix                          0
6e. zip                                    7
7. undergraduate (u) or graduate (g)       0
8. full-time (ft) or part-time (pt)        0
9. at-home or not-at-home               3177
university                                 0
year                                       0
Street_combined                          383
City                                       0
State                                      0
latitude_geocoded                       1445
longitude_geocoded                      1445
dtype: int64


In [ ]:
# Define the path where you want to save the CSV file
# Use forward slashes or raw string to avoid escape character issues
output_path = r"/content/drive/MyDrive/CS506_Project/API/Results"

# Create the directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Save each DataFrame to a separate CSV file with a descriptive name
df_test.to_csv(os.path.join(output_path, "api_results_student_addresses_2022_2023.csv"), index=False)

print("Files saved successfully to:")
print(output_path)

Files saved successfully to:
/content/drive/MyDrive/CS506_Project/API/Results
